In [1]:
# import roboschool
import gym
import numpy as np
import matplotlib.pyplot as plt
import os


import sys
sys.path.append('./asebo/')
from optimizers import Adam
from worker import worker, get_policy
from es import ES, aggregate_rollouts
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--env_name', type=str, default='Swimmer-v2')
# parser.add_argument('--steps', '-s', type=int, default=1000)
# parser.add_argument('--h_dim', '-hd', type=int, default=32)
# parser.add_argument('--start', '-st', type=int, default=0)
# parser.add_argument('--max_iter', '-it', type=int, default=1000)
# parser.add_argument('--seed', '-se', type=int, default=0)

# parser.add_argument('--k', '-k', type=int, default=70)
# parser.add_argument('--num_sensings', '-sn', type=int, default=100)
# parser.add_argument('--log', '-lg', type=int, default=0)
# parser.add_argument('--threshold', '-pc', type=float, default=0.995)
# parser.add_argument('--decay', '-dc', type=float, default=0.99)
# parser.add_argument('--learning_rate', '-lr', type=float, default=0.05)
# parser.add_argument('--filename', '-f', type=str, default='')
# parser.add_argument('--policy', '-po', type=str, default='Toeplitz') # Linear or Toeplitz

# parser.add_argument('--shift', '-sh', type=int, default=0)
# parser.add_argument('--min', '-mi', type=int, default=10)
# parser.add_argument('--sigma', '-si', type=float, default=0.1)

# args = parser.parse_args()
# params = vars(args)
params = {
# 'env_name': 'HalfCheetah-v2',
'env_name': 'InvertedDoublePendulum-v2',
'steps':1000,
'h_dim':32,
'start':0,
'max_iter':1000,
'seed':0,

'k':140,
'num_sensings':100,
'log':0,
'threshold':0.995,
'decay':0.99,
'learning_rate':0.05,
'filename':'',
'policy':'Linear', # Linear or Toeplitz
'shift':0,
'min':10,
'sigma':0.1
}

params['dir'] = params['env_name'] + params['policy'] + '_h' + str(params['h_dim']) + '_lr' + str(params['learning_rate']) + '_k' + str(params['k']) +'_' + params['filename']

# if not(os.path.exists('data/'+params['dir'])):
#     os.makedirs('data/'+params['dir'])
# os.chdir('data/'+params['dir'])

In [3]:
# run_asebo(params):
env = gym.make(params['env_name'])
params['ob_dim'] = env.observation_space.shape[0]
params['ac_dim'] = env.action_space.shape[0]

m = 0
v = 0

params['k'] += -1
params['alpha'] = 1

params['zeros'] = False
master = get_policy(params)

if params['log']:
    params['num_sensings'] = 4 + int(3 * np.log(master.N))

if params['k'] > master.N:
    params['k'] = master.N

n_eps = 0
n_iter = 1
ts_cumulative = 0
ts = []
rollouts = []
rewards = []
samples = []
alphas = []
G = []

In [4]:
while n_iter < params['max_iter']:

    params['n_iter'] = n_iter
    gradient, n_samples, timesteps = ES(params, master, G)
    ts_cumulative += timesteps
    ts.append(ts_cumulative)
    alphas.append(params['alpha'])

    if n_iter == 1:
        G = np.array(gradient)
    else:
        G *= params['decay']
        G = np.vstack([G, gradient])
    n_eps += 2 * n_samples
    rollouts.append(n_eps)
    gradient /= (np.linalg.norm(gradient) / master.N + 1e-8)

    update, m, v = Adam(gradient, m, v, params['learning_rate'], n_iter)

    master.update(update)
    test_policy = worker(params, master, np.zeros([1, master.N]), 0)
    reward = test_policy.rollout(train=False)
    rewards.append(reward)
    samples.append(n_samples)

    print('Iteration: %s, Rollouts: %s, Reward: %s, Alpha: %s, Samples: %s' %(n_iter, n_eps, reward, params['alpha'], n_samples))
    n_iter += 1

#     out = pd.DataFrame({'Rollouts': rollouts, 'Reward': rewards, 'Samples': samples, 'Timesteps': ts, 'Alpha': alphas})
#     out.to_csv('Seed%s.csv' %(params['seed']), index=False)        

Iteration: 1, Rollouts: 200, Reward: 54.210169479711396, Alpha: 1, Samples: 100
Iteration: 2, Rollouts: 400, Reward: 73.00875304454625, Alpha: 1, Samples: 100
Iteration: 3, Rollouts: 600, Reward: 129.15718557255772, Alpha: 1, Samples: 100
Iteration: 4, Rollouts: 800, Reward: 101.54912829598524, Alpha: 1, Samples: 100
Iteration: 5, Rollouts: 1000, Reward: 101.60285887754044, Alpha: 1, Samples: 100
Iteration: 6, Rollouts: 1200, Reward: 110.89084017794653, Alpha: 1, Samples: 100
Iteration: 7, Rollouts: 1400, Reward: 110.88014172093797, Alpha: 1, Samples: 100
Iteration: 8, Rollouts: 1600, Reward: 119.5609383151077, Alpha: 1, Samples: 100
Iteration: 9, Rollouts: 1800, Reward: 138.2669479063752, Alpha: 1, Samples: 100
Iteration: 10, Rollouts: 2000, Reward: 120.1780963289062, Alpha: 1, Samples: 100
Iteration: 11, Rollouts: 2200, Reward: 110.37251217614838, Alpha: 0.0, Samples: 100
Iteration: 12, Rollouts: 2220, Reward: 129.3063900306463, Alpha: 0.18088698266560205, Samples: 10
Iteration: 13, 

Iteration: 89, Rollouts: 3794, Reward: 183.97817568825226, Alpha: 0.0, Samples: 11
Iteration: 90, Rollouts: 3816, Reward: 193.52196048871363, Alpha: 0.0, Samples: 11
Iteration: 91, Rollouts: 3838, Reward: 240.60068241623506, Alpha: 0.0, Samples: 11
Iteration: 92, Rollouts: 3860, Reward: 240.7491574527508, Alpha: 0.0, Samples: 11
Iteration: 93, Rollouts: 3882, Reward: 315.0695575486315, Alpha: 0.0, Samples: 11
Iteration: 94, Rollouts: 3904, Reward: 286.46728958842175, Alpha: 0.0, Samples: 11
Iteration: 95, Rollouts: 3926, Reward: 231.60370045955185, Alpha: 0.0, Samples: 11
Iteration: 96, Rollouts: 3948, Reward: 138.2509776289712, Alpha: 0.0, Samples: 11
Iteration: 97, Rollouts: 3970, Reward: 138.3049340991001, Alpha: 0.0, Samples: 11
Iteration: 98, Rollouts: 3992, Reward: 138.29614368048271, Alpha: 0.0, Samples: 11
Iteration: 99, Rollouts: 4014, Reward: 138.31542168445756, Alpha: 0.0, Samples: 11
Iteration: 100, Rollouts: 4036, Reward: 174.11879655947237, Alpha: 0.0, Samples: 11
Iterati

Iteration: 189, Rollouts: 5994, Reward: 332.7646326952762, Alpha: 0.0, Samples: 11
Iteration: 190, Rollouts: 6016, Reward: 396.22214047459033, Alpha: 0.0, Samples: 11
Iteration: 191, Rollouts: 6038, Reward: 388.45178094184547, Alpha: 0.0, Samples: 11
Iteration: 192, Rollouts: 6060, Reward: 349.5272751470351, Alpha: 0.0, Samples: 11
Iteration: 193, Rollouts: 6082, Reward: 286.70632475795054, Alpha: 0.0, Samples: 11
Iteration: 194, Rollouts: 6104, Reward: 406.6031474853339, Alpha: 0.0, Samples: 11
Iteration: 195, Rollouts: 6126, Reward: 461.55518337083026, Alpha: 0.0, Samples: 11
Iteration: 196, Rollouts: 6148, Reward: 361.9873923584727, Alpha: 0.0, Samples: 11
Iteration: 197, Rollouts: 6170, Reward: 361.38990752295155, Alpha: 0.0, Samples: 11
Iteration: 198, Rollouts: 6192, Reward: 443.25534014195875, Alpha: 0.0, Samples: 11
Iteration: 199, Rollouts: 6214, Reward: 341.9889551958989, Alpha: 0.0, Samples: 11
Iteration: 200, Rollouts: 6236, Reward: 443.9532519755911, Alpha: 0.0, Samples: 1

Iteration: 288, Rollouts: 8172, Reward: 714.5983692408358, Alpha: 0.0, Samples: 11
Iteration: 289, Rollouts: 8194, Reward: 398.25482723534117, Alpha: 0.0, Samples: 11
Iteration: 290, Rollouts: 8216, Reward: 323.60457241052205, Alpha: 0.0, Samples: 11
Iteration: 291, Rollouts: 8238, Reward: 556.3260776488473, Alpha: 0.0, Samples: 11
Iteration: 292, Rollouts: 8260, Reward: 500.5281508784133, Alpha: 0.0, Samples: 11
Iteration: 293, Rollouts: 8282, Reward: 323.67530379685064, Alpha: 0.0, Samples: 11
Iteration: 294, Rollouts: 8304, Reward: 510.39052064224967, Alpha: 0.0, Samples: 11
Iteration: 295, Rollouts: 8326, Reward: 500.2030891886939, Alpha: 0.0, Samples: 11
Iteration: 296, Rollouts: 8348, Reward: 575.6114509798481, Alpha: 0.0, Samples: 11
Iteration: 297, Rollouts: 8370, Reward: 510.42432681049155, Alpha: 0.0, Samples: 11
Iteration: 298, Rollouts: 8392, Reward: 482.4395952627807, Alpha: 0.0, Samples: 11
Iteration: 299, Rollouts: 8414, Reward: 510.48432817931797, Alpha: 0.0, Samples: 1

Iteration: 387, Rollouts: 10350, Reward: 668.7495632407831, Alpha: 0.0, Samples: 11
Iteration: 388, Rollouts: 10372, Reward: 725.1539543651771, Alpha: 0.0, Samples: 11
Iteration: 389, Rollouts: 10394, Reward: 687.1460047162876, Alpha: 0.0, Samples: 11
Iteration: 390, Rollouts: 10416, Reward: 678.41643217242, Alpha: 0.0, Samples: 11
Iteration: 391, Rollouts: 10438, Reward: 696.5722389738112, Alpha: 0.0, Samples: 11
Iteration: 392, Rollouts: 10460, Reward: 668.6854687648798, Alpha: 0.0, Samples: 11
Iteration: 393, Rollouts: 10482, Reward: 696.9511091921605, Alpha: 0.0, Samples: 11
Iteration: 394, Rollouts: 10504, Reward: 724.7155854105296, Alpha: 0.0, Samples: 11
Iteration: 395, Rollouts: 10526, Reward: 762.1174253824003, Alpha: 0.0, Samples: 11
Iteration: 396, Rollouts: 10548, Reward: 771.7271490979335, Alpha: 0.0, Samples: 11
Iteration: 397, Rollouts: 10570, Reward: 799.6456641796376, Alpha: 0.0, Samples: 11
Iteration: 398, Rollouts: 10592, Reward: 902.538564542509, Alpha: 0.0, Samples

Iteration: 485, Rollouts: 12506, Reward: 696.29550540424, Alpha: 0.0, Samples: 11
Iteration: 486, Rollouts: 12528, Reward: 286.12540270670206, Alpha: 0.0, Samples: 11
Iteration: 487, Rollouts: 12550, Reward: 341.6688814531736, Alpha: 0.0, Samples: 11
Iteration: 488, Rollouts: 12572, Reward: 638.6990544107416, Alpha: 0.0, Samples: 11
Iteration: 489, Rollouts: 12594, Reward: 463.06496690062784, Alpha: 0.0, Samples: 11
Iteration: 490, Rollouts: 12616, Reward: 769.6760340218947, Alpha: 0.0, Samples: 11
Iteration: 491, Rollouts: 12638, Reward: 768.5152771111874, Alpha: 0.0, Samples: 11
Iteration: 492, Rollouts: 12660, Reward: 1087.5940533206774, Alpha: 0.0, Samples: 11
Iteration: 493, Rollouts: 12682, Reward: 751.8217062933363, Alpha: 0.0, Samples: 11
Iteration: 494, Rollouts: 12704, Reward: 1021.8697723793247, Alpha: 0.0, Samples: 11
Iteration: 495, Rollouts: 12726, Reward: 892.0053364069504, Alpha: 0.0, Samples: 11
Iteration: 496, Rollouts: 12748, Reward: 845.8586551683491, Alpha: 0.0, Sa

Iteration: 583, Rollouts: 14662, Reward: 836.8741198919275, Alpha: 0.0, Samples: 11
Iteration: 584, Rollouts: 14684, Reward: 826.7137383486156, Alpha: 0.0, Samples: 11
Iteration: 585, Rollouts: 14706, Reward: 882.5709235328026, Alpha: 0.0, Samples: 11
Iteration: 586, Rollouts: 14728, Reward: 799.3141430306703, Alpha: 0.0, Samples: 11
Iteration: 587, Rollouts: 14750, Reward: 790.6562740734481, Alpha: 0.0, Samples: 11
Iteration: 588, Rollouts: 14772, Reward: 771.116417658594, Alpha: 0.0, Samples: 11
Iteration: 589, Rollouts: 14794, Reward: 790.2554382040458, Alpha: 0.0, Samples: 11
Iteration: 590, Rollouts: 14816, Reward: 994.9556449847745, Alpha: 0.0, Samples: 11
Iteration: 591, Rollouts: 14838, Reward: 1060.4135216007571, Alpha: 0.0, Samples: 11
Iteration: 592, Rollouts: 14860, Reward: 1098.0257862953313, Alpha: 0.0, Samples: 11
Iteration: 593, Rollouts: 14882, Reward: 1134.9618984403514, Alpha: 0.0, Samples: 11
Iteration: 594, Rollouts: 14904, Reward: 1451.4751059882076, Alpha: 0.0, S

Iteration: 681, Rollouts: 16818, Reward: 2009.1318934939231, Alpha: 0.0, Samples: 11
Iteration: 682, Rollouts: 16840, Reward: 1831.578666626254, Alpha: 0.0, Samples: 11
Iteration: 683, Rollouts: 16862, Reward: 1720.3256258336485, Alpha: 0.0, Samples: 11
Iteration: 684, Rollouts: 16884, Reward: 1869.528265724424, Alpha: 0.0, Samples: 11
Iteration: 685, Rollouts: 16906, Reward: 1422.1575343592763, Alpha: 0.0, Samples: 11
Iteration: 686, Rollouts: 16928, Reward: 686.4178801418128, Alpha: 0.0, Samples: 11
Iteration: 687, Rollouts: 16950, Reward: 1757.8556233741524, Alpha: 0.0, Samples: 11
Iteration: 688, Rollouts: 16972, Reward: 1721.0542651855346, Alpha: 0.0, Samples: 11
Iteration: 689, Rollouts: 16994, Reward: 1748.3549191832692, Alpha: 0.0, Samples: 11
Iteration: 690, Rollouts: 17016, Reward: 2149.31870020806, Alpha: 0.0, Samples: 11
Iteration: 691, Rollouts: 17038, Reward: 2149.147239645806, Alpha: 0.0, Samples: 11
Iteration: 692, Rollouts: 17060, Reward: 2234.36186716006, Alpha: 0.0, 

Iteration: 779, Rollouts: 18974, Reward: 9321.72575596106, Alpha: 0.0, Samples: 11
Iteration: 780, Rollouts: 18996, Reward: 9321.90346302054, Alpha: 0.0, Samples: 11
Iteration: 781, Rollouts: 19018, Reward: 9323.075581074216, Alpha: 0.0, Samples: 11
Iteration: 782, Rollouts: 19040, Reward: 9322.858419087952, Alpha: 0.0, Samples: 11
Iteration: 783, Rollouts: 19062, Reward: 9322.968105376598, Alpha: 0.0, Samples: 11
Iteration: 784, Rollouts: 19084, Reward: 9322.872159551715, Alpha: 0.0, Samples: 11
Iteration: 785, Rollouts: 19106, Reward: 9323.796557914306, Alpha: 0.0, Samples: 11


KeyboardInterrupt: 

In [5]:
# np.save("./{}.npy".format(params['dir']), master.params)
# master.params=np.load("./{}.npy".format(params['dir']))
np.save("./data/{}_asebo.npy".format(params['dir']), master.params)
np.save("./data/{}_asebo_ts.npy".format(params['dir']), ts)
np.save("./data/{}_asebo_rs.npy".format(params['dir']), rewards)


In [6]:
test_policy = worker(params, master, np.zeros([1, master.N]), 0)
# reward = test_policy.rollout(train=False, render=True)

In [7]:
from gym.wrappers import Monitor
env = Monitor(gym.make(params['env_name']), './video', force=True)
env._max_episode_steps = params['steps']

In [8]:
def play(env, worker):

    state = env.reset()
    while 1:
        action = worker.policy.evaluate(state)
        action = np.clip(action, worker.env.action_space.low[0], worker.env.action_space.high[0])
        action = action.reshape(len(action), )
        state, reward, done, info = env.step(action)
        env.render()
        if done: 
            break

In [9]:
play(env, test_policy)

Creating offscreen glfw
Creating window glfw


SystemExit: 0

/Users/stevenyin/opt/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
